In [1]:
#Basic libraries
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#configure visualizations
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 15 , 8

#Train, test split and normalizing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer , Normalizer , scale, StandardScaler

#Cross Validation
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import validation_curve, GridSearchCV

import os, datetime as dt

In [2]:
#Feature selection
from sklearn.feature_selection import RFECV

#ML algo functions
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

#Metric
from sklearn.metrics import recall_score,roc_auc_score

seed = 19

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.precision', 3)

In [4]:
def info(df):
    size = df.shape[0]
    null = df.isnull().sum()
    nuni = train.nunique()
    dtype = df.dtypes
    missing_pct = null*100/size
    df_info  = pd.concat([dtype, missing_pct.round(2),nuni], axis=1, keys=['type','missing','nunique'])
    df_info.sort_values (['missing'],ascending=False,inplace=True) #null_val.sort_values(ascending=False).round(2)
    return df_info

In [5]:
folder = r"G:\Study material\Data Science\Datasets\Loan_HackerEarth\\"
train_file = "train_indessa.csv"
test_file = "test_indessa.csv"

In [6]:
#Data read
train = pd.read_csv(os.path.join(folder,train_file))
train.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,9 years,OWN,28700.0,Source Verified,n,NaN,debt_consolidation,Debt consolidation,349xx,FL,33.88,0.0,1.0,50.0,75.0,14.0,1.0,22515.0,73.1,28.0,f,1173.84,0.0,0.0,0.0,0.0,74.0,INDIVIDUAL,NaN,26th week,0.0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,< 1 year,MORTGAGE,65000.0,Source Verified,n,NaN,home_improvement,Home improvement,209xx,MD,3.64,0.0,1.0,NaN,NaN,6.0,0.0,7624.0,23.2,13.0,w,83.95,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,2 years,OWN,45000.0,Not Verified,n,NaN,debt_consolidation,Debt consolidation,447xx,OH,18.42,0.0,0.0,NaN,NaN,5.0,0.0,10877.0,31.2,19.0,w,56.47,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,10+ years,RENT,105000.0,Not Verified,n,> My goal is to obtain a loan to pay off my high credit cards and get out of debt within 3 years.<br>,debt_consolidation,Debt consolidation,221xx,VA,14.97,0.0,2.0,46.0,NaN,10.0,0.0,13712.0,55.5,21.0,f,4858.62,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,135th week,0.0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,RENT,52000.0,Verified,n,NaN,credit_card,refi,900xx,CA,20.16,0.0,0.0,NaN,NaN,11.0,0.0,35835.0,76.2,27.0,w,2296.41,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,96th week,0.0,0.0,47159.0,47033.0,0


In [7]:
train.shape

(532428, 45)

In [8]:
id_col = train.pop('member_id')
target = train.pop('loan_status')

In [9]:
df_info = info(train)
df_info.T

,verification_status_joint,desc,mths_since_last_record,mths_since_last_major_derog,mths_since_last_delinq,batch_enrolled,total_rev_hi_lim,tot_cur_bal,tot_coll_amt,emp_title,revol_util,collections_12_mths_ex_med,title,last_week_pay,initial_list_status,revol_bal,total_acc,total_rec_late_fee,total_rec_int,recoveries,pub_rec,acc_now_delinq,application_type,collection_recovery_fee,loan_amnt,open_acc,funded_amnt,funded_amnt_inv,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths
type,object,object,float64,float64,float64,object,float64,float64,float64,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,float64,object,float64,int64,float64,int64,float64,object,float64,object,object,object,object,float64,object,object,object,object,object,float64,float64,float64
missing,99.9,85.8,84.6,75,51.2,16,7.89,7.89,7.89,5.79,0.05,0.02,0.02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
nunique,3,70638,122,162,147,104,14697,251640,8081,190125,1265,9,39693,98,2,63459,126,4073,243149,14024,28,8,2,12617,1370,73,1370,6442,2,535,7,35,12,6,33988,3,2,14,917,51,4058,26,23


In [10]:
target.value_counts(normalize=True)

0    0.764
1    0.236
Name: loan_status, dtype: float64

In [11]:
train.dtypes.value_counts()

float64    23
object     18
int64       2
dtype: int64

In [12]:
df_reset = train.copy()

In [ ]:
#train = df_reset_copy()

## Data Description  
Total columns 43 + member id + target
Possible Leakage: Funded_amt, funded_amnt_inv

In [13]:
#Treating missing values
df_info[df_info['missing'] > 40].shape

(5, 3)

In [14]:
df_info[df_info['missing'] > 40]

,type,missing,nunique
verification_status_joint,object,99.94,3
desc,object,85.80,70638
mths_since_last_record,float64,84.58,122
mths_since_last_major_derog,float64,75.02,162
mths_since_last_delinq,float64,51.19,147


In [15]:
drop_missing = list(df_info[df_info['missing'] > 40].index) #Too high missing values

In [16]:
drop_missing

['verification_status_joint',
 'desc',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'mths_since_last_delinq']

In [17]:
train.drop(drop_missing, axis =1 , inplace = True)

In [18]:
cat_col = train.dtypes[train.dtypes == "object"].index

In [19]:
#Fill NA by No_value
train[cat_col] = train[cat_col].fillna('missing')

In [36]:
num_missing = list(df_info[(df_info['missing'] > 0) & (df_info['type'] != 'object')].index)

In [37]:
num_missing_after_drop = [col for col in num_missing if col not in drop_missing]

In [38]:
num_missing_after_drop

['total_rev_hi_lim',
 'tot_cur_bal',
 'tot_coll_amt',
 'revol_util',
 'collections_12_mths_ex_med']

In [39]:
for col in num_missing_after_drop:
    avg = np.mean(train[col].dropna())
    print(col," : ",avg)

total_rev_hi_lim  :  32080.572918535796
tot_cur_bal  :  139554.1107918862
tot_coll_amt  :  213.5622216694126
revol_util  :  55.0571891660288
collections_12_mths_ex_med  :  0.014299320162379564


In [40]:
for col in num_missing_after_drop:
    avg = np.mean(train[col].dropna())
    train[col].fillna(avg,inplace=True)

In [41]:
#Check status
clean_info = info(train)
clean_info.T

,loan_amnt,total_rec_late_fee,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,recoveries,funded_amnt,collection_recovery_fee,collections_12_mths_ex_med,application_type,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,inq_last_6mths,delinq_2yrs,dti,addr_state,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,title,zip_code,total_rev_hi_lim
type,int64,float64,float64,float64,float64,float64,float64,object,float64,float64,int64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,object,float64,object,object,float64,object,object,object,object,object,float64,object,object,object,object,object,float64
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
nunique,1370,4073,73,28,63459,1266,126,2,243149,14024,1370,12617,10,2,98,8,8082,251641,23,26,4058,51,6442,2,105,535,7,35,190126,12,6,33988,3,2,14,39694,917,14698


### Data cleaned

In [43]:
clean_info[clean_info['type'] == 'object'].T

,initial_list_status,application_type,last_week_pay,addr_state,term,batch_enrolled,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,pymnt_plan,purpose,title,zip_code
type,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
nunique,2,2,98,51,2,105,7,35,190126,12,6,3,2,14,39694,917


In [44]:
from sklearn.preprocessing import LabelEncoder

In [47]:
le = LabelEncoder()
bin_cols = []
for col in cat_col:
    if train[col].nunique() <= 2:
        bin_cols.append(col)
        le.fit(train[col])
        train[col] = le.transform(train[col])                

print(bin_cols)

['term', 'pymnt_plan', 'initial_list_status', 'application_type']


In [50]:
#too many unique values
high_nunique_cols = ['emp_title','title']

In [53]:
#drop one column from each categorical feature to solve multi collinearity issue
#Drop column with max value count
dummies_2_drop = []
dummies_col = []
for col in cat_col:
    if ((col not in bin_cols) and (col not in high_nunique_cols)):
        dummies_col.append(col)
        dummies_2_drop.append(col+"_"+train[col].value_counts().index[0])
        
dummies_2_drop

['batch_enrolled_ ',
 'grade_B',
 'sub_grade_B3',
 'emp_length_10+ years',
 'home_ownership_MORTGAGE',
 'verification_status_Source Verified',
 'purpose_debt_consolidation',
 'zip_code_945xx',
 'addr_state_CA',
 'last_week_pay_13th week']

In [54]:
dummies_col

['batch_enrolled',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'zip_code',
 'addr_state',
 'last_week_pay']

In [63]:
dummies_list = pd.DataFrame()
for col in dummies_col:
    dummies_list = pd.concat([dummies_list,pd.get_dummies(train[col],prefix=col)],axis=1)

In [59]:
dummies_list.shape

(532428, 1248)

In [64]:
dummies_list.columns

Index(['batch_enrolled_ ', 'batch_enrolled_BAT1104812', 'batch_enrolled_BAT1135695', 'batch_enrolled_BAT1184694', 'batch_enrolled_BAT1273836', 'batch_enrolled_BAT1327206', 'batch_enrolled_BAT1467036', 'batch_enrolled_BAT1521494', 'batch_enrolled_BAT1575727', 'batch_enrolled_BAT1586599',
       ...
       'last_week_pay_82th week', 'last_week_pay_83th week', 'last_week_pay_87th week', 'last_week_pay_8th week', 'last_week_pay_91th week', 'last_week_pay_92th week', 'last_week_pay_95th week', 'last_week_pay_96th week', 'last_week_pay_9th week', 'last_week_pay_NAth week'], dtype='object', length=1248)

In [65]:
train = pd.concat([train,dummies_list],axis=1)

In [66]:
train.drop(dummies_2_drop,inplace=True,axis=1)

In [67]:
train.shape

(532428, 2524)

In [ ]:
corr = df.corr()
plt.subplots( figsize =( 16 , 10 ) )
sns.heatmap(corr)
plt.show()

In [ ]:
df['term_new'] = df['term'].apply(lambda x: int(x.split()[0]))

In [ ]:
df['term_new'].unique()

In [ ]:
df['batch_enrolled'].unique()

In [ ]:
df['annual_inc'].head()

In [ ]:
lb_make = LabelEncoder()
df["batch"] = lb_make.fit_transform(df["batch_enrolled"])

In [ ]:
df['grade'].unique()

In [ ]:
grade_dict = {'A': 1, 'B': 2, 'C':3, 'D':4,'E':5,'F':6,'G':7}

In [ ]:
df['grade_new'] = df['grade'].apply(lambda x: grade_dict[x])

In [ ]:
df['grade_new'].unique()

In [ ]:
df['sub_grade'].unique()

In [ ]:
int(str(grade_dict[df.loc[0,'sub_grade'][0]])+df.loc[0,'sub_grade'][1])

In [ ]:
df['sub_grade_new'] = df['sub_grade'].apply(lambda x: int(str(grade_dict[x[0]])+x[1]))

In [ ]:
df['sub_grade_new'].unique()

In [ ]:
df['emp_length'].unique()

In [ ]:
emp_len = {'< 1 year' : 0, '1 year' : 1,'2 years' : 2, '10+ years' :10, '5 years' : 5, '7 years' : 7, \
           '4 years' : 4, '3 years' : 3, '6 years' : 6,  '8 years' : 8, '9 years' : 9}

In [ ]:
df['emp_new'] = df['emp_length'].apply(lambda x: emp_len[x])

In [ ]:
df['emp_new'].unique()

In [ ]:
df['home_ownership'].unique()

In [ ]:
# create dummies
home = pd.get_dummies(df.home_ownership,prefix = 'H')

In [ ]:
df['verification_status'].unique()

In [ ]:
veri = pd.get_dummies(df.verification_status,prefix = 'V')

In [ ]:
df['pymnt_plan'].unique()

In [ ]:
df['pymnt_new'] = df['pymnt_plan'].apply(lambda x: 0 if x == 'n' else 1)

In [ ]:
df['pymnt_new'].unique()

In [ ]:
df['purpose'].unique()

In [ ]:
purpose = pd.get_dummies(df.purpose,prefix = 'P')

In [ ]:
df['zip_code'][0][:3]

In [ ]:
df['zip_new'] = df['zip_code'].apply(lambda x: int(x[:3]))

In [ ]:
df['zip_new'].head()

In [ ]:
df['addr_state'].head()

In [ ]:
df["state"] = lb_make.fit_transform(df["addr_state"])
df[["addr_state", "state"]].head(4)

In [ ]:
df['initial_list_status'].unique()

In [ ]:
df["ini_list"] = df['initial_list_status'].apply(lambda x: 0 if x == 'f' else 1)

In [ ]:
df['application_type'].unique()

In [ ]:
df["app"] = df['application_type'].apply(lambda x: 0 if x == 'INDIVIDUAL' else 1)

In [ ]:
df['app'].unique()

In [ ]:
df['last_week_pay'] = df['last_week_pay'].apply(lambda x: '0th week' if x == 'NAth week' else x)

In [ ]:
import re
df['last_wk'] = df['last_week_pay'].apply(lambda x: int(re.findall(r'\d{1,4}',x)[0]))

In [ ]:
#Get everything together
#home, veri, purpose
y = df['loan_status']
cols = ['loan_amnt', 'int_rate', 'annual_inc', \
        'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', \
        'pub_rec', 'revol_bal', 'revol_util', 'total_acc', \
        'total_rec_int', 'total_rec_late_fee', 'recoveries', \
        'collection_recovery_fee', 'collections_12_mths_ex_med', \
        'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', \
        'term_new', 'grade_new', 'sub_grade_new', 'emp_new', \
        'zip_new', 'state', 'batch', 'ini_list', 'last_wk']

In [ ]:
X = pd.concat([df[cols],home,veri,purpose],axis=1)

In [ ]:
len(X.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
scaler = Normalizer()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Feature importance
model = DecisionTreeClassifier(random_state = seed)
model.fit(X_train_scaled,y_train)
imp = pd.DataFrame(model.feature_importances_ ,columns = [ 'Importance' ] , \
                       index = X.columns)
imp = imp.sort_values( [ 'Importance' ] , ascending = True )
imp[:].plot( kind = 'barh' )
plt.show()

In [ ]:
model = GradientBoostingClassifier()
model.fit(X_train_scaled,y_train)
y_predicted = model.predict(X_test_scaled)
print("Train score: ",model.score(X_train_scaled,y_train))
print("Test score: ",model.score(X_test_scaled,y_test))